In [1]:
from get_results import *

### Tokenizers check

In [2]:
from huggingface_hub import login
login()

In [3]:
### set this args with argparse, now hardcoded
GEMMA=True

if GEMMA == True:
    N_CONTEXT = 1024 # number of context tokens to consider
    N_BATCHES = 128 # number of batches to consider
    TOTAL_BATCHES = 20 

    RELEASE = 'gemma-2b-res-jb'
    BASE_MODEL = "google/gemma-2b"
    FINETUNE_MODEL = 'shahdishank/gemma-2b-it-finetune-python-codes'
    DATASET_NAME = "ctigges/openwebtext-gemma-1024-cl"
    hook_part = 'post'
    layer_num = 6
else:
    N_CONTEXT = 128 # number of context tokens to consider
    N_BATCHES = 128 # number of batches to consider
    TOTAL_BATCHES = 100 

    RELEASE = 'gpt2-small-res-jb'
    BASE_MODEL = "gpt2-small"
    FINETUNE_MODEL = 'pierreguillou/gpt2-small-portuguese'
    DATASET_NAME = "Skylion007/openwebtext"
    hook_part = 'pre'
    layer_num = 6

SAE_HOOK = f'blocks.{layer_num}.hook_resid_{hook_part}'

In [4]:
tokenizer_cfg = TokenizerComparisonConfig(BASE_MODEL, FINETUNE_MODEL)

In [5]:
compare_tokenizers(tokenizer_cfg)

PYTHONPATH: C:\Edu\Mech_Interp\SAE_merging\SAE-Tuning-Merging
DATAPATH: C:\Edu\Mech_Interp\SAE_merging\SAE-Tuning-Merging\data


,Metric,Value
0,Base Tokenizer Size,256000.000000
1,Fine-tune Tokenizer Size,256000.000000
2,Keys only in Base,1.000000
3,Keys only in Fine-tune,1.000000
4,Keys with Mismatched Token IDs,0.000000
5,Good Tokens in Base (%),99.999609
6,Good Tokens in Fine-tune (%),99.999609


### Activations storage

In [2]:
### set this args with argparse, now hardcoded
GEMMA=False

if GEMMA == True:
    N_CONTEXT = 1024 # number of context tokens to consider
    N_BATCHES = 8 # number of batches to consider
    TOTAL_BATCHES = 20 

    RELEASE = 'gemma-2b-res-jb'
    BASE_MODEL = "google/gemma-2b"
    FINETUNE_MODEL = 'shahdishank/gemma-2b-it-finetune-python-codes'
    DATASET_NAME = "ctigges/openwebtext-gemma-1024-cl"
    hook_part = 'post'
    layer_num = 6
else:
    N_CONTEXT = 128 # number of context tokens to consider
    N_BATCHES = 8 # number of batches to consider
    TOTAL_BATCHES = 40 

    RELEASE = 'gpt2-small-res-jb'
    BASE_MODEL = "gpt2-small"
    FINETUNE_MODEL = 'pierreguillou/gpt2-small-portuguese'
    DATASET_NAME = "Skylion007/openwebtext"
    hook_part = 'pre'
    layer_num = 6

SAE_HOOK = f'blocks.{layer_num}.hook_resid_{hook_part}'

In [3]:
activation_cfg = ActivationStoringConfig(BASE_MODEL, FINETUNE_MODEL, DATASET_NAME, 
                                         N_CONTEXT, N_BATCHES, TOTAL_BATCHES, 
                                         layer_num, SAE_HOOK)

In [9]:
result_dict = get_activations_for_base_and_ft(activation_cfg)

PYTHONPATH: C:\Edu\Mech_Interp\SAE_merging\SAE-Tuning-Merging
DATAPATH: C:\Edu\Mech_Interp\SAE_merging\SAE-Tuning-Merging\data


Loaded pretrained model gpt2-small into HookedTransformer
Run name: 2048-L1-0.001-LR-0.0003-Tokens-2.000e+06
n_tokens_per_buffer (millions): 0.02048
Lower bound: n_contexts_per_buffer (millions): 0.00016
Total training steps: 488
Total wandb updates: 48
n_tokens_per_feature_sampling_window (millions): 1048.576
n_tokens_per_dead_feature_window (millions): 524.288
We will reset the sparsity calculation 0 times.
Number tokens in sparsity calculation window: 8.19e+06


Loaded pretrained model gpt2-small into HookedTransformer


In [10]:
base_act = torch.load(result_dict['base_act_path'])
finetune_act = torch.load(result_dict['finetune_act_path'])

base_act.shape, finetune_act.shape

C:\Users\Tarassaco\AppData\Local\Temp\ipykernel_9728\683010509.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  base_act = torch.load(result_dict['base_act_path'])
C:\Use

(torch.Size([320, 128, 768]), torch.Size([320, 128, 768]))